# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
import joblib 
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice.aci import AciWebservice
from azureml.core.webservice import Webservice
import azureml.core


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto_ml_hr_analytics'

experiment=Experiment(ws, experiment_name)


In [3]:
for ds in ws.datasets:
    print(ds)

HR_Analytics


In [4]:
data = ws.datasets["HR_Analytics"]
data

{
  "source": [
    "('workspaceblobstore', 'UI/02-07-2021_122258_UTC/aug_train.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "ad8ae88d-b694-45df-a5a0-a40d4349755f",
    "name": "HR_Analytics",
    "version": 1,
    "workspace": "Workspace.create(name='quick-starts-ws-137675', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-137675')"
  }
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
try:
    cluster_project = ComputeTarget(workspace=ws, name="aml-compute-capstone")
    print("Cluster exists")
except:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cluster_project = ComputeTarget.create(ws, "aml-compute-capstone", config)

cluster_project.wait_for_completion()

Cluster exists


In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "task": "classification", 
    "primary_metric": "accuracy",
    "training_data": data,
    "label_column_name": "target",
    "n_cross_validations": 5,
    "compute_target": cluster_project
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output=True)

Running on remote.
No run_configuration provided, running on aml-compute-capstone with default configuration
Running on remote compute: aml-compute-capstone
Parent Run ID: AutoML_f0ad48f8-eb49-4ded-ae3f-db5055e453ba

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_auto_run, best_auto_model = remote_run.get_output()
print(best_auto_run)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: auto_ml_hr_analytics,
Id: AutoML_f0ad48f8-eb49-4ded-ae3f-db5055e453ba_21,
Type: azureml.scriptrun,
Status: Completed)


In [12]:
print(best_auto_run)

Run(Experiment: auto_ml_hr_analytics,
Id: AutoML_f0ad48f8-eb49-4ded-ae3f-db5055e453ba_21,
Type: azureml.scriptrun,
Status: Completed)


In [13]:
best_auto_model._final_estimator

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('19',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7facc37f9da0>),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=0.6933333333333332,
                                                                               importance_typ...
                                                                                   min_sample

In [14]:

joblib.dump(best_auto_model, filename="./automl_best_model.joblib")

['./automl_best_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
model = best_auto_run.register_model("best_model", model_path="outputs/model.pkl")

In [16]:
best_auto_run.download_file('outputs/scoring_file_v_1_0_0.py', 'automl_inf/score.py')
best_auto_run.download_file('outputs/model.pkl', 'automl_inf/model.pkl')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
inference_config = InferenceConfig(entry_script='automl_inf/score.py')

TODO: In the cell below, print the logs of the web service and delete the service

In [18]:
service = Model.deploy(ws, "best-model-service", [model], inference_config, overwrite=True)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [22]:
import json
data = {"data":
        [
          {
            "city_development_index": 0.94,
            "gender": "Male",
            "relevent_experience": "Has relevent experience",
            "enrolled_university": "Full time course",
            "education_level": "Graduate",
            "major_discipline": "STEM",
            "experience": ">20",
            "company_size": "<10",
            "company_type": "Pvt Ltd",
            "last_new_job": 4,
            "training_hours": 30
          },
          {
            "city_development_index": 0.559,
            "gender": "Female",
            "relevent_experience": "No relevent experience",
            "enrolled_university": "no_enrollment",
            "education_level": "Masters",
            "major_discipline": "No Major",
            "experience": 1,
            "company_size": "100-500",
            "company_type": "Pvt Ltd",
            "last_new_job": "never",
            "training_hours": 7
          },
      ]
    }
input_data = json.dumps(data)

In [24]:
output = service.run(input_data)
output

'{"result": [0.0, 1.0]}'

In [25]:
service.delete()